In [1]:
from pandas import DataFrame
import pickle
import networkx as nx
import pandas as pd

In [2]:
def check_status(ban_status: bool):
        if ban_status == True:
            return 'Cheater'
        return 'Legit'

In [3]:
def generate_vac_graph_only_processed():
    G = nx.Graph()
    with open("data/bloomData.pkl", "rb") as dataPickle:
        df_full_data: DataFrame = pickle.load(dataPickle, fix_imports=True)
    
    processedIDs = set[str]([id for id in df_full_data.index])
    
    for _, row in df_full_data.iterrows():
        current_id = row['steamid']
        vac_banned = row['VACBanned']
        
        status = check_status(vac_banned)
        
        G.add_node(current_id, status = status)
        
        for friend in row["friendsList"]:
            if friend['steamid'] in processedIDs:
                
                friend_ban_status = df_full_data[df_full_data['steamid'] == friend['steamid']]['VACBanned'].iloc[0]
                friend_status = check_status(friend_ban_status)
                G.add_node(friend['steamid'], status = friend_status)
                G.add_edge(current_id, friend['steamid'], edge_type='friend')
    print('Done')
    nx.write_gexf(G, "./data/tennative_graph_only_processed_new.gexf")
    return G


In [4]:
G = generate_vac_graph_only_processed()

Done
